In [ ]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import math

In [ ]:
# read the csv file with SLC, soil and landuse info
slc = pd.read_csv('/home/paulc600/SMM/HYPE_geospatial/SLC_frac.csv')

In [ ]:
slc

In [ ]:
# remove soil code from index
slc = slc.reset_index()
slc.loc[-1] = slc.columns
slc.sort_index(inplace=True)

In [ ]:
slc.drop('index', axis=1, inplace=True)

In [ ]:
slc

In [ ]:
# Remove uncecessary rows
removed_slc = slc.iloc[:3]

In [ ]:
removed_slc

In [ ]:
# Transpose the DataFrame
slc_transposed = removed_slc.transpose().reset_index(drop=True)

In [ ]:
# Set the first row as the index header
slc_transposed.set_axis(slc_transposed.iloc[0], axis=1, inplace=True)
slc_transposed = slc_transposed[1:]

In [ ]:
# re order columns
new_order=['COMID\SLC', 'Landuse code', 'soil code']

In [ ]:
# Rearrange the columns in the DataFrame
slc_transposed = slc_transposed[new_order]

In [ ]:
# Convert all columns to int by truncating decimals
slc_transposed = slc_transposed.applymap(lambda x: int(float(x)) if isinstance(x, str) and '.' in x else int(x))

In [ ]:
#Rename the columns
slc_transposed.rename(columns={'COMID\SLC':'SLC' , 'soil code':'SOIL TYPE', 'Landuse code':'LULC'}, inplace=True)   

In [ ]:
slc_transposed

In [ ]:
# Replace the values of 'COMID\SLC' with a sequence of numbers
slc_transposed['SLC'] = range(1, len(slc_transposed) + 1)

In [ ]:
slc_transposed

In [ ]:
#amount of rows for the rest of the data
array_rows=53

In [ ]:
# Numbers to fill the columns with
fill_numbers = [0, 0, 0, 1, 0, 0, 2.296, 3, 0.091, 0.493, 2.296]

In [ ]:
# Create a dictionary with column names and fill numbers
extra_data = {'Main crop cropid': [fill_numbers[0]] * array_rows,
        'Second crop cropid': [fill_numbers[1]] * array_rows,
        'Crop rotation group': [fill_numbers[2]] * array_rows,
        'Vegetation type': [fill_numbers[3]] * array_rows,
        'Special class code': [fill_numbers[4]] * array_rows,
        'Tile depth': [fill_numbers[5]] * array_rows,
        'Stream depth': [fill_numbers[6]] * array_rows,
        'Number of soil layers': [fill_numbers[7]] * array_rows,
        'Soil layer depth 1': [fill_numbers[8]] * array_rows,
        'Soil layer depth 2': [fill_numbers[9]] * array_rows,
        'Soil layer depth 3': [fill_numbers[10]] * array_rows}

In [ ]:
# Create the DataFrame
full_data = pd.DataFrame(extra_data)

In [ ]:
full_data

In [ ]:
# Merge the two DataFrames
final_geoclass = pd.merge(slc_transposed, full_data, left_index=True, right_index=True)

In [ ]:
final_geoclass

In [ ]:
# Output merged DataFrame to a .txt file
output_file = '/home/paulc600/local/HYPE Inputs/GeoClass.txt'

In [ ]:
# Add commented lines
commented_lines = [
"""! MODIS landcover													
! Add legend (raster value) and discription													
!	original legend (raster_value)	description											
!	1	Evergreen Needleleaf Forests 											
!	2	Evergreen Broadleaf Forests 											
!	3	Deciduous Needleleaf Forests 											
!	4	Deciduous Broadleaf Forests 											
!	5	Mixed Forests 											
!	6	Closed Shrublands 											
!	7	Open Shrublands 											
!	8	Woody Savannas 											
!	9	Savannas 											
!	10	Grasslands 											
!	11	Permanent Wetlands 											
!	12	Croplands 											
!	13	Urban and Built-up Lands 											
!	14	Cropland/ Natural Vegetation Mosmcs 											
!	15	Permanent Snow and Ice 											
!	16	Barren 											
!	17	Water Bodies 											
!													
!													
!													
! ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------													
!	SoilGrid V1												
!		original legend (raster_value)	description										
!	 C 	1	 clay										
!	 SIC 	2	 silty clay										
!	 SC 	3	 sandy clay										
!	 CL 	4	 clay loam										
!	 SICL 	5	 silty clay loam										
!	 SCL 	6	 sandy clay loam										
!	 L 	7	 loam										
!	 SIL 	8	 silty loam										
!	 SL 	9	 sandy loam										
!	 SI 	10	 silt										
!	 LS 	11	 loamy sand										
!	 S 	12	  sand										
!													
!													
! ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	
!          SLC	LULC	SOIL TYPE	Main crop cropid	Second crop cropid	Crop rotation group	Vegetation type	Special class code	Tile depth	Stream depth	Number of soil layers	Soil layer depth 1	Soil layer depth 2	Soil layer depth 3"""
]

In [ ]:
# Open the file in write mode
with open(output_file, 'w') as file:
    # Write the commented lines
    for line in commented_lines:
        file.write(line + '\n')

In [ ]:
with open(output_file, 'a') as file:
        final_geoclass.to_csv(file, sep='\t', index=False, header=False, line_terminator='\n')